In [97]:
import os
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY')
)

df=pd.read_pickle('cleaned_quests_with_glossary.pkl')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15530 entries, 0 to 15634
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              15530 non-null  int64 
 1   expansion       15530 non-null  object
 2   title_en        15530 non-null  object
 3   objective_en    14152 non-null  object
 4   description_en  14157 non-null  object
 5   progress_en     9613 non-null   object
 6   completion_en   14963 non-null  object
 7   title_uk        6116 non-null   object
 8   objective_uk    5276 non-null   object
 9   description_uk  5273 non-null   object
 10  progress_uk     4536 non-null   object
 11  completion_uk   6012 non-null   object
 12  cat             15530 non-null  object
 13  glossary        15530 non-null  object
dtypes: int64(1), object(13)
memory usage: 1.8+ MB


In [98]:
def generate_request_json(row):
    return {
        "original": {
            "quest": {
                "title": row["title_en"],
                "objective": row["objective_en"],
                "description": row["description_en"],
                "progress": row["progress_en"],
                "completion": row["completion_en"]
            }
        },
        "glossary": {
            "glossary": row["glossary"]
        }
    }

def generate_response_json(row):
    return {
        "translation": {
            "quest": {
                "title": row["title_uk"],
                "objective": row["objective_uk"],
                "description": row["description_uk"],
                "progress": row["progress_uk"],
                "completion": row["completion_uk"]
            }
        }
    }


In [ ]:
TRANSLATION_SYSTEM_PROMPT = """You are a professional translator specializing in MMORPG game localization, responsible for translating World of Warcraft content from English to Ukrainian. 
You will receive a JSON file containing an "original" section with the object to translate. 
Your task is to respond with a JSON containing single section, "translation," that replicates the structure of the "original" section, replacing source text with translated values while maintaining all keys.
Use your knowledge of the Warcraft universe to ensure accuracy and immersion.

### **Guidelines**:

1. **Preserve Structure**:
   - The output JSON must retain the exact structure of the "original" section.
   - Ensure all keys remain the same while replacing values with their Ukrainian translations.

2. **Maintain Placeholders**:
   - Always replace name/race/class placeholders in the original text with curly-bracedanslated placeholders with shortened Ukrainian grammatical case after a semicolon. For `<name>`, `<race>`, `<class>` use `{ім'я:н}`, `{раса:д}`, `{клас:к}` with correct grammatical case afet semicolon.

3. **Adapt for Gender**:
   - Translate text to be suitable for both genders. For placeholders with gender-specific variations (e.g., `<his/her>`, `<priest/priestess>`) and translated words that have different gender forms use the format `{стать:male:female}` (e.g., `{стать:його:її}`, `{стать:жрець:жриця}`).
   - Gender placeholders should contain full words, so don't break words with placeholder: for example, instead of кмітлив{стать:ий:а} use {стать:кмітливий:кмітлива}
   - Gender placeholders can contain more that one word. If there's several consequent gender-specific placeholders or they are divided with small word - put tham in single placeholder. Fo example: instead of "дуже {стать:достойний:достойна} та {стать:спроможний:спроможна}" use "дуже {стать:достойний та спроможний:достойна та спроможна}"

4. **Formal and Informal Tone**:
   - **Quest `objective` field**: Translate using a formal tone, where "you" becomes "ви" and "your" becomes "ваш."
   - **Quest `description`, `progress`, and `completion` fields**: By default, translate these fields using an informal tone, where "you" becomes "ти" and "your" becomes "твій.". If text describes environment/interaction or NPC speaks to player politely using "sir", "lady" etc - use formal tone.

5. **Preserve Descriptive Formatting**:
   - Retain all text enclosed in angle brackets (`<>`) that describes the environment, character reactions, or similar narrative elements. Translate the content within the brackets but preserve their formatting.

6. **Maintain Lore and Style**:
   - Ensure the translation captures the epic, immersive tone of World of Warcraft.
   - Use established Warcraft terminology and lore-consistent language in Ukrainian.

7. **Glossaries and terminology**:
   - Always strictly refer to the translation glossary. Always use terms' translations from glossary, even if other more commonly used or recognized translations are available.
   - Use consistent terminology throughout all translations. Adhere strictly to any provided glossaries, ensuring uniformity in translating terms such as character names, locations, and game-specific items.

8. **Readability**:
   - Ensure translations are natural, engaging, and free from awkward phrasing.
   - Text should feel native and immersive for Ukrainian-speaking players.

---

### **Input Example**:

```json
{
  "original": {
    "quest": {
      "title": "Get rid of these boars",
      "objective": "Kill 12 boars and return to John at Scarlet Monastery in Silverpine Forest.",
      "description": "Reports have come in that wild boars trample our crops. We thought that other adventurer had dealt with this, <class>!\n\nI suggest that you to teach them a lesson as proof of your devotion.",
      "progress": "Have you clear our fields of wild boars?",
      "completion": "<John nods to you.>\n\nWe are pleased that you have prooved your devition, <name>. Surely now that you have made our point clear multiple times, their senseless attacks upon our crops will cease?"
    }
  }
}
```

---

### **Output Example**:

```json
{
  "translation": {
    "quest": {
      "title": "Позбудься цих вепрів",
      "objective": "Вбийте 12 вепрів і поверніться до Джона в Багряний Монастир, що в Срібнохвойному пралісі.",
      "description": "Надходять повідомлення про диких вепрів, що топчуть наші посіви. Ми думали, що інші шукачі пригод вже розібралися з цим, {клас:к}!\n\nРаджу тобі провчити їх як доказ твоєї відданості.",
      "progress": "Ти вже {стать:очистив:очистила} наші поля від диких вепрів?",
      "completion": "<Джон киває вам.>\n\nМи задоволені, що ти {стать:довів:довела} свою відданість, {ім'я:к}. Тепер, коли ми вже неодноразово довели свою точку зору, може їхні безглузді напади на наші посіви припиняться?"
    }
  }
}
```

---
"""

In [107]:
filtered_quests = df[df['description_uk'].str.contains(r'\:р}', na=False)]
filtered_quests.head()

,id,expansion,title_en,objective_en,description_en,progress_en,completion_en,title_uk,objective_uk,description_uk,progress_uk,completion_uk,cat,glossary
314,334,classic,Package for Thurman,Go to the Larson Clothiers in the Stormwind Ma...,My son Thurman is an apprentice at the Larson ...,Are you here to buy clothes?,"Oh, blast! I thought forgetting my kit would f...",Пакунок для Турмана,"Відправляйтесь в ""сукна Ларсона"", що у квартал...","Мій син Турман — учень в ""сукнах Ларсона"", що ...",Вітаю! Бажаєте щось купити?,"Ой, ле! Я думав, що, забувши інструменти, я зм...",Eastern Kingdoms/Stormwind City,"[{'en': 'Stormwind', 'uk': 'Штормовій'}, {'en'..."
812,861,classic,The Hunter's Way,Bring 4 Flatland Prowler Claws to Melor Stoneh...,"You are eager to explore, I can tell. I too ha...",Greetings. There is an air about you that tell...,Skorn Whitecloud is a wise <race>. He has hunt...,Шлях мисливця,Принесіть 4 пазури рівнинного скрадача Мелоров...,"А ти {стать:охочий:охоча} до мандрівок, я можу...",Вітаю. Твоя присутність багато про що говорить...,Скорн Біла Хмара - мудрий таурен. Він полював ...,Kalimdor/Mulgore,"[{'en': 'Mulgore', 'uk': 'Мулґор'}, {'en': 'Th..."
1298,1498,classic,Path of Defense,Bring 5 Singed Scales to Uzzek at Far Watch Po...,"As a warrior, one of our most crucial tasks is...","Do you have the scales, <name>? Unless you can...",Ah. I see you have defeated the thunder lizard...,Мистецтво захисту,Принесіть 5 припалених лусок Узеку на Далекост...,"Одна з твоїх найважливіших задач, як {клас:р}....","{стать:Приніс:Принесла} луску, {ім'я:к}? Якщо ...","А-а-а, бачу, ти {стать:поборов:поборола} громо...",Classes/Warrior,"[{'en': 'The Horde', 'uk': 'Орда'}, {'en': 'Ba..."
1315,1516,classic,Call of Earth,Bring 2 Felstalker Hooves to Canaga Earthcalle...,"The time is now, young <class>. You've grown s...",Patience is earth's greatest virtue. The earth...,Excellent. Your success shows that you are pre...,Поклик землі,Принесіть 2 копита скверноловів Каназі Землезо...,"Час прийшов, {стать:юний:юна} {клас:к}. Твої с...",Терпіння — найбільша чеснота землі. Земля є св...,"Чудово. Твій успіх доводить, що ти {стать:гото...",Classes/Shaman,"[{'en': 'Valley of Trials', 'uk': 'долина Випр..."
1372,1679,classic,Muren Stormpike,Speak with Muren Stormpike.,"Hello, warrior! You show a lot of promise, but...",None,"Ah, <name>. I heard the warriors of Dun Morogh...",Мурен Бурешпиль,Поговоріть з Муреном Бурешпилем.,"Вітаю, {клас:к}! Ти непогано себе {стать:прояв...",None,"А, {ім'я}. Чув я щось про тебе від воїнів Дун-...",Classes/Warrior,"[{'en': 'Dun Morogh', 'uk': 'Дун-Морог'}, {'en..."


In [108]:
quest_row = filtered_quests.iloc[0]
quest_row['glossary']

[{'en': 'Stormwind', 'uk': 'Штормовій'},
 {'en': 'mage', 'uk': 'маг/магиня'},
 {'en': 'apprentice', 'uk': 'учень'},
 {'en': 'Thurman Schneider', 'uk': 'Турман Шнайдер'}]

In [116]:
messages = [{'role': 'system', 'content': TRANSLATION_SYSTEM_PROMPT}, {'role': 'user', 'content': str(generate_request_json(quest_row))}]

response = client.chat.completions.create(
  # model="gpt-4o",
  # model="gpt-4o-mini",
  model="ft:gpt-4o-mini-2024-07-18:personal:classicua-trained-3:AXaw8vp9",
  # model="gpt-3.5-turbo",
  messages=messages,
  response_format={"type":"json_object"},
  temperature=0.5,
  top_p=0.7
)

# print(response.choices[0].message)
import json
json_response = response.choices[0].message.content
json.loads(json_response)['translation']

{'quest': {'title': 'Посилка для Турмана',
  'objective': 'Відправляйтесь до Ларсонів в Штормовій, в квартал магів, і віддайте Турману Шнайдеру його швейний набір.',
  'description': "Мій син Турман — учень у Ларсонів, в кварталі магів. Сьогодні він так поспішав, що забув свої ножиці та голки. Я знаю, що такий {клас:к}, як ти, має важливіші справи, але без своїх інструментів Турман не зможе виконувати свої обов'язки!\n\nБудь ласка, {ім'я}. Можеш віднести моєму синові його швейний набір? Ларсони — одна з двох швейних крамниць в кварталі магів. Вони розташовані далі всередину, біля вежі магів.",
  'progress': 'Ви тут, щоб купити одяг?',
  'completion': 'Ой, чорт! Я думав, що забувши свій набір, я позбудуся роботи. Тепер мені, мабуть, доведеться допомагати Ларсонам з шиттям...\n\nНу, що ж. Я думаю, що розваги прийдуть пізніше.'}}

In [103]:
import json
json_response = response.choices[0].message.content
json.loads(json_response)['translation']

{'quest': {'title': 'Посилка для Турмана',
  'objective': 'Іди до Ларсонів у Штормовій, в Магічному Кварталі, і передай Турману Шнайдеру його Набір для Шиття.',
  'description': "Мій син Турман - учень у Ларсонів, в Магічному Кварталі. Сьогодні він поспішав і забув свої ножиці та голки. Я знаю, що такий чудовий {клас:к}, як ти, має важливі справи, але без своїх інструментів Турман не зможе виконувати свою учнівську роботу!\n\nБудь ласка, {ім'я:н}. Ти можеш віднести набір для шиття мого сина йому? Ларсонів - один з двох магазинів одягу в Магічному Кварталі - це той, що глибше, біля Магічної Вежі.",
  'progress': 'Ти тут, щоб купити одяг?',
  'completion': 'О, чорт! Я думав, що забувши свій набір, звільнюся від роботи. Тепер, мабуть, мені доведеться допомагати Ларсонам з їхнім шиттям...\n\nНу, що ж. Розваги прийдуть пізніше, я гадаю.'}}

In [ ]:
translated = df[df['title_uk'].notnull()]
untranslated = df[df['title_uk'].isnull()]
translated.head()

In [117]:
def construct_prompt_completion(quest_row):
    messages = [{'role': 'system', 'content': TRANSLATION_SYSTEM_PROMPT}, {'role': 'user', 'content': str(generate_request_json(quest_row))}, {'role': 'assistant', 'content': str(generate_response_json(quest_row))}]
    return {"messages": messages}

jsonl_data = translated.apply(construct_prompt_completion, axis=1).tolist()

In [111]:
# %pip install tiktoken
import tiktoken

def num_tokens_from_string(string: str, encoding_name:str="cl100k_base") -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(str(jsonl_data))

13691292

In [118]:
# Meticulously chosen and refined translations with various cases for pre-training:
pre_training_quest_ids = [(22, "classic"), (60, "classic"),(188, "classic"),(230, "classic"),(334, "classic"),(365, "classic"),(575, "classic"),(595, "classic"),(663, "wrath"),(752, "classic"),(753, "classic"),(884, "classic"),(902, "classic"),(1140, "classic"),(1718, "classic"),(2278, "classic"),(2279, "classic"),(2280, "classic"),(2358, "wrath"),(2947, "classic"),(2949, "classic"),(2978, "classic"),(3741, "classic"),(3942, "classic"),(4602, "classic"),(4603, "classic"),(5210, "classic"),(5943, "classic"),(9393, "tbc"),(9402, "tbc"),(9488, "tbc"),(9523, "tbc"),(9558, "tbc"),(9595, "tbc"),(9616, "tbc"),(9646, "tbc"),(9649, "tbc"),(9696, "tbc"),(9797, "tbc"),(10065, "tbc"),(77643, "sod")]
# Some more data for pre-training:
additional_pre_training_quest_ids = [(2, "classic"),(5, "classic"),(6, "classic"),(7, "classic"),(8, "classic"),(9, "classic"),(10, "classic"),(11, "classic"),(12, "classic"),(13, "classic"),(14, "classic"),(15, "classic"),(17, "classic"),(18, "classic"),(19, "classic"),(20, "classic"),(21, "classic"),(23, "classic"),(24, "classic"),(25, "classic"),(26, "classic"),(28, "classic"),(33, "classic"),(34, "classic"),(35, "classic"),(36, "classic"),(37, "classic"),(38, "classic"),(39, "classic"),(40, "classic"),(45, "classic"),(46, "classic"),(47, "classic"),(52, "classic"),(54, "classic"),(55, "classic"),(56, "classic"),(57, "classic"),(58, "classic"),(59, "classic"),(62, "classic"),(64, "classic"),(65, "classic"),(66, "classic"),(67, "classic"),(68, "classic"),(69, "classic"),(70, "classic"),(71, "classic"),(72, "classic"),(74, "classic"),(75, "classic"),(76, "classic"),(85, "classic"),(85, "classic"),(86, "classic"),(87, "classic"),(88, "classic"),(89, "classic")]

filter_condition = pd.DataFrame([*pre_training_quest_ids, *additional_pre_training_quest_ids], columns=["id", "expansion"])
pre_training_quests = translated.merge(filter_condition, on=["id", "expansion"])

In [119]:
jsonl_data = pre_training_quests.apply(construct_prompt_completion, axis=1).tolist()
num_tokens_from_string(str(jsonl_data))

227569

In [120]:
import json

with open('pre_training_data.jsonl', 'w', encoding='utf-8') as f:
    for entry in jsonl_data[:80]:
        json.dump(entry, f, ensure_ascii=False)
        f.write('\n')

with open('pre_training_validation.jsonl', 'w', encoding='utf-8') as f:
    for entry in jsonl_data[80:]:
        json.dump(entry, f, ensure_ascii=False)
        f.write('\n')

In [121]:
with open('test.json', 'w', encoding='utf-8') as f:
    json.dump(generate_request_json(untranslated.iloc[3]), f, ensure_ascii=False)

trolls and dwarfs language...